In [16]:
#correct weekly plan
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import numpy as np
import re
from math import pow

# Load the dataset
file_path = 'nutrition_cf - Sheet4.csv'
nutrition_data = pd.read_csv(file_path)

# Function to filter dataset based on multiple allergies, region, and category using regex
def filter_dataset(df, allergies, region_pattern, category):
    if allergies == ["no-allergies"]:
        allergy_condition = df['Allergy'].apply(lambda x: True)
    else:
        allergy_condition = df['Allergy'].apply(lambda x: not any(re.search(allergy, x, re.IGNORECASE) for allergy in allergies))

    if category == 'veg':
        category_condition = df['Category'].str.lower() == 'veg'
    elif category == 'eggetarian':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg'])
    elif category == 'non-veg':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg', 'non-veg'])
    else:
        category_condition = False

    if region_pattern != "Null":
        region_filter = df['Region'].apply(lambda x: re.search(region_pattern, x, re.IGNORECASE) is not None)
    else:
        region_filter = df['Region'].apply(lambda x: True)
        #return df[allergy_condition & category_condition]

    return df[allergy_condition & region_filter & category_condition]

## Function to set target nutritional values based on BMI and gender
def set_target_values(bmi, gender):
    if gender.lower() == 'male':
        if bmi < 18.5:
            # Underweight
            return {
                'breakfast': {'Proteins': 15, 'Carbohydrates': 70, 'Fats': 15, 'Fiber': 8, 'Energy(kcal)': 500},
                'lunch': {'Proteins': 18, 'Carbohydrates': 75, 'Fats': 18, 'Fiber': 8, 'Energy(kcal)': 600},
                'snacks': {'Proteins': 10, 'Carbohydrates': 40, 'Fats': 10, 'Fiber': 4, 'Energy(kcal)': 300},
                'dinner': {'Proteins': 20, 'Carbohydrates': 90, 'Fats': 18, 'Fiber': 10, 'Energy(kcal)': 650},
                'appetizers': {'Proteins': 7, 'Carbohydrates': 35, 'Fats': 7, 'Fiber': 5, 'Energy(kcal)': 250},
            }
        elif bmi < 24.9:
            # Normal weight
            return {
                'breakfast': {'Proteins': 12, 'Carbohydrates': 60, 'Fats': 12, 'Fiber': 6, 'Energy(kcal)': 450},
                'lunch': {'Proteins': 13, 'Carbohydrates': 60, 'Fats': 13, 'Fiber': 6, 'Energy(kcal)': 450},
                'snacks': {'Proteins': 7, 'Carbohydrates': 35, 'Fats': 7, 'Fiber': 3, 'Energy(kcal)': 250},
                'dinner': {'Proteins': 17, 'Carbohydrates': 80, 'Fats': 13, 'Fiber': 8, 'Energy(kcal)': 550},
                'appetizers': {'Proteins': 5, 'Carbohydrates': 30, 'Fats': 5, 'Fiber': 3, 'Energy(kcal)': 200},
            }
        else:
            # Overweight
            return {
                'breakfast': {'Proteins': 10, 'Carbohydrates': 50, 'Fats': 10, 'Fiber': 5, 'Energy(kcal)': 400},
                'lunch': {'Proteins': 12, 'Carbohydrates': 50, 'Fats': 12, 'Fiber': 5, 'Energy(kcal)': 400},
                'snacks': {'Proteins': 5, 'Carbohydrates': 30, 'Fats': 5, 'Fiber': 2, 'Energy(kcal)': 200},
                'dinner': {'Proteins': 15, 'Carbohydrates': 70, 'Fats': 10, 'Fiber': 7, 'Energy(kcal)': 500},
                'appetizers': {'Proteins': 4, 'Carbohydrates': 25, 'Fats': 4, 'Fiber': 2, 'Energy(kcal)': 150},
            }
    else:
        # Similar logic for female, adjust values as needed
        if bmi < 18.5:
            # Underweight
            return {
                'breakfast': {'Proteins': 14, 'Carbohydrates': 65, 'Fats': 14, 'Fiber': 7, 'Energy(kcal)': 480},
                'lunch': {'Proteins': 16, 'Carbohydrates': 70, 'Fats': 16, 'Fiber': 7, 'Energy(kcal)': 550},
                'snacks': {'Proteins': 9, 'Carbohydrates': 38, 'Fats': 9, 'Fiber': 4, 'Energy(kcal)': 280},
                'dinner': {'Proteins': 18, 'Carbohydrates': 85, 'Fats': 16, 'Fiber': 9, 'Energy(kcal)': 600},
                'appetizers': {'Proteins': 6, 'Carbohydrates': 33, 'Fats': 6, 'Fiber': 4, 'Energy(kcal)': 230},
            }
        elif bmi < 24.9:
            # Normal weight
            return {
                'breakfast': {'Proteins': 12, 'Carbohydrates': 60, 'Fats': 12, 'Fiber': 6, 'Energy(kcal)': 450},
                'lunch': {'Proteins': 13, 'Carbohydrates': 60, 'Fats': 13, 'Fiber': 6, 'Energy(kcal)': 450},
                'snacks': {'Proteins': 7, 'Carbohydrates': 35, 'Fats': 7, 'Fiber': 3, 'Energy(kcal)': 250},
                'dinner': {'Proteins': 17, 'Carbohydrates': 80, 'Fats': 13, 'Fiber': 8, 'Energy(kcal)': 550},
                'appetizers': {'Proteins': 5, 'Carbohydrates': 30, 'Fats': 5, 'Fiber': 3, 'Energy(kcal)': 200},
            }
        else:
            # Overweight
            return {
                'breakfast': {'Proteins': 10, 'Carbohydrates': 50, 'Fats': 10, 'Fiber': 5, 'Energy(kcal)': 400},
                'lunch': {'Proteins': 12, 'Carbohydrates': 50, 'Fats': 12, 'Fiber': 5, 'Energy(kcal)': 400},
                'snacks': {'Proteins': 5, 'Carbohydrates': 30, 'Fats': 5, 'Fiber': 2, 'Energy(kcal)': 200},
                'dinner': {'Proteins': 15, 'Carbohydrates': 70, 'Fats': 10, 'Fiber': 7, 'Energy(kcal)': 500},
                'appetizers': {'Proteins': 4, 'Carbohydrates': 25, 'Fats': 4, 'Fiber': 2, 'Energy(kcal)': 150},
            }


# User inputs for allergies, region, and category
'''user_allergies_input = input("Enter allergies separated by commas (e.g., mushroom, peanut): ")
user_region_pattern = input("Enter region pattern (regex) (e.g., North): ")
user_category = input("Enter category (e.g., veg, non-veg, eggetarian): ")
user_gender = input("Enter your gender: ")
user_height = float(input("Enter your height: "))
user_weight = int(input("Enter your weight: "))'''

user_allergies_input = 'no-allergies'
user_region_pattern = "continental"
user_category = "non-veg"
user_gender = "male"
user_height = 1.85
user_weight = 80

print("User preference\nAllergy : "+user_allergies_input+"\nRegion : "+user_region_pattern+"\nCategory : "+user_category+"\n")

# Parsing user allergies input into a list
user_allergies = [allergy.strip() for allergy in user_allergies_input.split(',')]
if user_allergies == ['']:
    user_allergies = ["no-allergies"]

# Define associativity rules
associativity_rules = {
    #'PITA BREAD (WHEAT)': ['Hummus', 'TABBOULEH'],
    '1': ['2'],
    '3': ['2', '4', '8'],
    '5': ['4', '8'],
    '6': ['7'],
    '11': ['10','12'],
    '14': ['13'],
    '9': ['10','12']
    # Add more rules as needed
}

bmi = (user_weight)/pow(user_height,2)
target_values = set_target_values(bmi, user_gender)
#print("BMI:",bmi)

target_breakfast = target_values['breakfast']
target_lunch = target_values['lunch']
target_snacks = target_values['snacks']
target_dinner = target_values['dinner']
target_appetizers = target_values['appetizers']

# Function to check if combined nutritional requirements are met
def check_combined_nutritional_requirements(food1, food2, target_nutrients):
    combined_nutrients = food1[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']] + food2[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']]
    meets_requirements = all(combined_nutrients <= pd.Series(target_nutrients))
    if(combined_nutrients <= pd.Series(target_nutrients)).all():
        return meets_requirements

def check_nutritional_requirements(food, target_nutrients,):
    combined_nutrients = food[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']]
    meets_requirements = all(combined_nutrients <= pd.Series(target_nutrients))
    if(combined_nutrients <= pd.Series(target_nutrients)).all():
        return meets_requirements

def recommend_food(df, meal_type, target_nutrients, num_recommendations=27):
    meal_data = df[df['Type'].str.contains(meal_type, case=False, na=False)]
    features = ['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)', 'Carbon Footprint(kg CO2e)']

    X = meal_data[features].values
    y = meal_data['Food'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    knn = NearestNeighbors(n_neighbors=num_recommendations,metric= 'cosine')
    knn.fit(X_train_scaled, y_train)

    target_values = np.array([target_nutrients['Proteins'], target_nutrients['Carbohydrates'],
                              target_nutrients['Fats'], target_nutrients['Fiber'],
                              target_nutrients['Energy(kcal)'], 0]).reshape(1, -1)
    target_values_scaled = scaler.transform(target_values)

    distances, indices = knn.kneighbors(target_values_scaled)

    recommended_foods = meal_data.iloc[indices[0]]
    #print(recommended_foods.head(2))
    return recommended_foods.head(num_recommendations)

def divide_by_serving(food):
  serving = food['Serving']
  food['Serving_Numbers'] =  int(serving[0])
  food[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']] = food[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']].div(food['Serving_Numbers'], axis=0)
  food = food.drop(['Serving_Numbers'], axis=0)
  return food

def divide_by_serving_combo(food, assoc_food, target_nutrients):
    serving = food['Serving']
    assoc_serving = assoc_food['Serving']

    new_food = food.copy()
    new_assoc_food = assoc_food.copy()

    new_food['Serving_Numbers'] = int(serving[0])
    new_assoc_food['Serving_Numbers_assoc'] = int(assoc_serving[0])

    new_food[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']] = new_food[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']].div(new_food['Serving_Numbers'], axis=0)
    new_assoc_food[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']] = new_assoc_food[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']].div(new_assoc_food['Serving_Numbers_assoc'], axis=0)

    if check_combined_nutritional_requirements(new_food, new_assoc_food, target_nutrients):
        return [new_food['Food'], new_food['Energy(kcal)'], new_assoc_food["Food"], new_assoc_food["Energy(kcal)"]]

    new_food = new_food.drop(['Serving_Numbers'], axis=0)
    new_assoc_food = new_assoc_food.drop(['Serving_Numbers_assoc'], axis=0)

    return []


# Function to print recommendations with associative rules
def get_weekly_plan(recommended_foods, associative_rules, valid_associations, target_nutrients):
    assoc_food_present = []
    weekly_plan = []

    for index, row in recommended_foods.iterrows():
        food_item = row['Food']
        associativity = str(row['Associativity'])
        calorie = row['Energy(kcal)']
        protein = row['Proteins']
        carbs = row['Carbohydrates']
        fats = row['Fats']
        fiber = row['Fiber']

        # Split associativity values if they are combined (e.g., '2,8')
        associativity_values = [value.strip() for value in associativity.split(',')]
        associated_foods = []
        combined_calories = calorie
        combined_proteins = protein
        combined_carbs = carbs
        combined_fats = fats
        combined_fiber = fiber

        for value in associativity_values:
            if value in valid_associations and value in associative_rules:
                if value in ['11', '9', '5', '3', '1']:
                    associated_food_items = nutrition_data[nutrition_data['Associativity'].isin(associative_rules[value])]
                else:
                    associated_food_items = recommended_foods[recommended_foods['Associativity'].isin(associative_rules[value])]
                if value != '14':
                    associated_food_items = associated_food_items[~associated_food_items['Food'].isin(assoc_food_present)]
                if not associated_food_items.empty:
                    for _, assoc_row in associated_food_items.iterrows():
                        if check_combined_nutritional_requirements(row, assoc_row, target_nutrients):
                            assoc_food_present.append(assoc_row['Food'])
                            associated_foods.append(assoc_row['Food'])
                            combined_calories += assoc_row['Energy(kcal)']
                            combined_proteins += assoc_row['Proteins']
                            combined_carbs += assoc_row['Carbohydrates']
                            combined_fats += assoc_row['Fats']
                            combined_fiber += assoc_row['Fiber']
                            break
                        else:
                            food_df = divide_by_serving_combo(row, assoc_row, target_nutrients)
                            if isinstance(food_df, pd.DataFrame) and not food_df.empty:
                                assoc_food_present.append(assoc_row['Food'])
                                associated_foods.append(food_df.iloc[0]['Food'])
                                combined_calories += food_df.iloc[0]['Energy(kcal)']
                                combined_proteins += food_df.iloc[0]['Proteins']
                                combined_carbs += food_df.iloc[0]['Carbohydrates']
                                combined_fats += food_df.iloc[0]['Fats']
                                combined_fiber += food_df.iloc[0]['Fiber']
                                break

        if associated_foods:
            weekly_plan.append({
                #'Food': food_item,
                #'Associations': ', '.join(associated_foods),
                'Proteins': combined_proteins,
                'Carbohydrates': combined_carbs,
                'Fats': combined_fats,
                'Fiber': combined_fiber,
                'Energy(kcal)': combined_calories
            })
        elif '0' in associativity_values:
            if check_nutritional_requirements(row, target_nutrients):
                weekly_plan.append({
                    #'Food': food_item,
                    #'Associations': '',
                    'Proteins': protein,
                    'Carbohydrates': carbs,
                    'Fats': fats,
                    'Fiber': fiber,
                    'Energy(kcal)': calorie
                })
            else:
                df = divide_by_serving(row)
                if isinstance(df, pd.DataFrame) and not df.empty:
                    if check_nutritional_requirements(df.iloc[0], target_nutrients):
                        weekly_plan.append({
                            #'Food': df.iloc[0]['Food'],
                            #'Associations': '',
                            'Proteins': df.iloc[0]['Proteins'],
                            'Carbohydrates': df.iloc[0]['Carbohydrates'],
                            'Fats': df.iloc[0]['Fats'],
                            'Fiber': df.iloc[0]['Fiber'],
                            'Energy(kcal)': df.iloc[0]['Energy(kcal)']
                        })

    return pd.DataFrame(weekly_plan[:7])


# Filtering the dataset based on user preferences
try:
    filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)
    filtered_data_null = filter_dataset(nutrition_data, user_allergies, "Null", user_category)
    try:
        recommended_breakfast = recommend_food(filtered_data, 'Breakfast', target_breakfast)
    except:
        recommended_breakfast = recommend_food(filtered_data_null, 'Breakfast', target_breakfast)

    try:
        recommended_snacks = recommend_food(filtered_data, 'Snacks', target_snacks)
    except:
        recommended_snacks = recommend_food(filtered_data_null, 'Snacks', target_snacks)

    try:
        recommended_appetizers = recommend_food(filtered_data, 'Appetizer', target_appetizers)
    except:
        recommended_appetizers = recommend_food(filtered_data_null, 'Appetizer', target_appetizers)

finally:
    recommended_lunch = recommend_food(filtered_data, 'Lunch', target_lunch)
    recommended_dinner = recommend_food(filtered_data, 'Dinner', target_dinner)


'''# Recommend food items for each meal type
print("Recommended Breakfast :"+ str(target_breakfast['Energy(kcal)']))
print_recommendations_with_associative_rules(recommended_breakfast, associativity_rules, ['0', '1', '3', '5','9','14'], target_breakfast)

print("\nRecommended Appetizers for Lunch :" + str(target_appetizers['Energy(kcal)']))
print_recommendations_with_associative_rules(recommended_appetizers, associativity_rules, ['0', '6'], target_appetizers)

print("\nRecommended Lunch :"+ str(target_lunch['Energy(kcal)']))
print_recommendations_with_associative_rules(recommended_lunch, associativity_rules, ['0', '1', '3', '5','14'], target_lunch)

print("\nRecommended Dinner :"+ str(target_dinner['Energy(kcal)']))
print_recommendations_with_associative_rules(recommended_dinner, associativity_rules, ['0', '1', '3', '5','14'], target_dinner)

print("\nRecommended Snacks :"+ str(target_snacks['Energy(kcal)']))
print_recommendations_with_associative_rules(recommended_snacks, associativity_rules, ['0', '11'], target_snacks)'''

breakfast_plan = get_weekly_plan(recommended_breakfast, associativity_rules, ['0', '1', '3', '5','9','14'], target_breakfast)
snacks_plan = get_weekly_plan(recommended_snacks, associativity_rules, ['0', '11'], target_snacks)
appetizers_plan = get_weekly_plan(recommended_appetizers, associativity_rules, ['0', '6'], target_appetizers)
lunch_plan = get_weekly_plan(recommended_lunch, associativity_rules,['0', '1', '3', '5','14'], target_lunch)
dinner_plan = get_weekly_plan(recommended_dinner, associativity_rules,['0', '1', '3', '5','14'], target_dinner)

# Format into a weekly plan
weekly_plan = [
     breakfast_plan,lunch_plan,dinner_plan,snacks_plan,appetizers_plan
]

# Print the weekly plan
for meals in weekly_plan:
        print(f"{meals}")
    

def calculate_accuracy(weekly_plan, target_values):
    errors = []
    meal_types = ['breakfast', 'lunch', 'snacks', 'dinner', 'appetizers']
    
    for meal_type, meal_plan in (meal_types, weekly_plan):
        target_nutrients = target_values[meal_type]
        total_error = 0
        count = 0
        for day_plan in meal_plan:
            for _, food in day_plan.iterrows():
                for nutrient, target_value in target_nutrients.items():
                    error = abs(food[nutrient] - target_value) / target_value
                    total_error += error
                    count += 1
        if count > 0:
            avg_error = total_error / count
            errors.append(avg_error)
    
    overall_error = np.mean(errors) if errors else 0
    accuracy = (1 - overall_error) * 100
    return accuracy

# Example usage
accuracy = calculate_accuracy(weekly_plan, target_values)
print(f'Accuracy of the recommendations: {accuracy:.2f}%')


    

User preference
Allergy : no-allergies
Region : continental
Category : non-veg

   Proteins  Carbohydrates  Fats  Fiber  Energy(kcal)
0       0.0            2.0   0.0    0.0             7
1       4.0           23.0   4.0    6.0           146
2       8.0           15.0  11.0    6.0           195
3       9.0           29.0   8.0    4.0           222
4       7.0           52.0   6.0    6.0           311
5       8.0           20.0  11.0    6.0           220
6       2.0            8.0   5.0    1.0            84
   Proteins  Carbohydrates  Fats  Fiber  Energy(kcal)
0       9.0           34.0   4.0    6.0           202
1       1.0            6.0   3.0    1.0            59
2       2.0           19.0  10.0    2.0           154
3       4.0           12.0   5.0    2.0           110
4       1.0            1.0   8.0    1.0            79
5       6.0           15.0   6.0    5.0           160
6       2.0           43.0   5.0    3.0           236
   Proteins  Carbohydrates  Fats  Fiber  Energy(kcal)
0 

ValueError: too many values to unpack (expected 2)

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import numpy as np
import re
from math import pow

# Load the dataset
file_path = 'nutrition_cf - Sheet4.csv'
nutrition_data = pd.read_csv(file_path)

# Function to filter dataset based on multiple allergies, region, and category using regex
def filter_dataset(df, allergies, region_pattern, category):
    if allergies == ["no-allergies"]:
        allergy_condition = df['Allergy'].apply(lambda x: True)
    else:
        allergy_condition = df['Allergy'].apply(lambda x: not any(re.search(allergy, x, re.IGNORECASE) for allergy in allergies))

    if category == 'veg':
        category_condition = df['Category'].str.lower() == 'veg'
    elif category == 'eggetarian':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg'])
    elif category == 'non-veg':
        category_condition = df['Category'].str.lower().isin(['veg', 'egg', 'non-veg'])
    else:
        category_condition = False

    if region_pattern != "Null":
        region_filter = df['Region'].apply(lambda x: re.search(region_pattern, x, re.IGNORECASE) is not None)
    else:
        region_filter = df['Region'].apply(lambda x: True)

    return df[allergy_condition & region_filter & category_condition]

# Function to set target nutritional values based on BMI and gender
def set_target_values(bmi, gender):
    if gender.lower() == 'male':
        if bmi < 18.5:
            return {
                'breakfast': {'Proteins': 15, 'Carbohydrates': 70, 'Fats': 15, 'Fiber': 8, 'Energy(kcal)': 500},
                'lunch': {'Proteins': 18, 'Carbohydrates': 75, 'Fats': 18, 'Fiber': 8, 'Energy(kcal)': 600},
                'snacks': {'Proteins': 10, 'Carbohydrates': 40, 'Fats': 10, 'Fiber': 4, 'Energy(kcal)': 300},
                'dinner': {'Proteins': 20, 'Carbohydrates': 90, 'Fats': 18, 'Fiber': 10, 'Energy(kcal)': 650},
                'appetizers': {'Proteins': 7, 'Carbohydrates': 35, 'Fats': 7, 'Fiber': 5, 'Energy(kcal)': 250},
            }
        elif bmi < 24.9:
            return {
                'breakfast': {'Proteins': 12, 'Carbohydrates': 60, 'Fats': 12, 'Fiber': 6, 'Energy(kcal)': 450},
                'lunch': {'Proteins': 13, 'Carbohydrates': 60, 'Fats': 13, 'Fiber': 6, 'Energy(kcal)': 450},
                'snacks': {'Proteins': 7, 'Carbohydrates': 35, 'Fats': 7, 'Fiber': 3, 'Energy(kcal)': 250},
                'dinner': {'Proteins': 17, 'Carbohydrates': 80, 'Fats': 13, 'Fiber': 8, 'Energy(kcal)': 550},
                'appetizers': {'Proteins': 5, 'Carbohydrates': 30, 'Fats': 5, 'Fiber': 3, 'Energy(kcal)': 200},
            }
        else:
            return {
                'breakfast': {'Proteins': 10, 'Carbohydrates': 50, 'Fats': 10, 'Fiber': 5, 'Energy(kcal)': 400},
                'lunch': {'Proteins': 12, 'Carbohydrates': 50, 'Fats': 12, 'Fiber': 5, 'Energy(kcal)': 400},
                'snacks': {'Proteins': 5, 'Carbohydrates': 30, 'Fats': 5, 'Fiber': 2, 'Energy(kcal)': 200},
                'dinner': {'Proteins': 15, 'Carbohydrates': 70, 'Fats': 10, 'Fiber': 7, 'Energy(kcal)': 500},
                'appetizers': {'Proteins': 4, 'Carbohydrates': 25, 'Fats': 4, 'Fiber': 2, 'Energy(kcal)': 150},
            }
    else:
        if bmi < 18.5:
            return {
                'breakfast': {'Proteins': 14, 'Carbohydrates': 65, 'Fats': 14, 'Fiber': 7, 'Energy(kcal)': 480},
                'lunch': {'Proteins': 16, 'Carbohydrates': 70, 'Fats': 16, 'Fiber': 7, 'Energy(kcal)': 550},
                'snacks': {'Proteins': 9, 'Carbohydrates': 38, 'Fats': 9, 'Fiber': 4, 'Energy(kcal)': 280},
                'dinner': {'Proteins': 18, 'Carbohydrates': 85, 'Fats': 16, 'Fiber': 9, 'Energy(kcal)': 600},
                'appetizers': {'Proteins': 6, 'Carbohydrates': 33, 'Fats': 6, 'Fiber': 4, 'Energy(kcal)': 230},
            }
        elif bmi < 24.9:
            return {
                'breakfast': {'Proteins': 12, 'Carbohydrates': 60, 'Fats': 12, 'Fiber': 6, 'Energy(kcal)': 450},
                'lunch': {'Proteins': 13, 'Carbohydrates': 60, 'Fats': 13, 'Fiber': 6, 'Energy(kcal)': 450},
                'snacks': {'Proteins': 7, 'Carbohydrates': 35, 'Fats': 7, 'Fiber': 3, 'Energy(kcal)': 250},
                'dinner': {'Proteins': 17, 'Carbohydrates': 80, 'Fats': 13, 'Fiber': 8, 'Energy(kcal)': 550},
                'appetizers': {'Proteins': 5, 'Carbohydrates': 30, 'Fats': 5, 'Fiber': 3, 'Energy(kcal)': 200},
            }
        else:
            return {
                'breakfast': {'Proteins': 10, 'Carbohydrates': 50, 'Fats': 10, 'Fiber': 5, 'Energy(kcal)': 400},
                'lunch': {'Proteins': 12, 'Carbohydrates': 50, 'Fats': 12, 'Fiber': 5, 'Energy(kcal)': 400},
                'snacks': {'Proteins': 5, 'Carbohydrates': 30, 'Fats': 5, 'Fiber': 2, 'Energy(kcal)': 200},
                'dinner': {'Proteins': 15, 'Carbohydrates': 70, 'Fats': 10, 'Fiber': 7, 'Energy(kcal)': 500},
                'appetizers': {'Proteins': 4, 'Carbohydrates': 25, 'Fats': 4, 'Fiber': 2, 'Energy(kcal)': 150},
            }

# Define Precision@k function
def precision_at_k(recommended, relevant, k):
    recommended_items = set(recommended[:k])
    relevant_items = set(relevant)
    if len(recommended_items) == 0:
        return 0.0
    return len(recommended_items.intersection(relevant_items)) / k

# Define Average Precision (AP) function
def average_precision(recommended, relevant):
    ap = 0.0
    num_relevant = 0
    for i, item in enumerate(recommended):
        if item in relevant:
            num_relevant += 1
            ap += num_relevant / (i + 1)
    if num_relevant == 0:
        return 0.0
    return ap / num_relevant

# Define Mean Average Precision (MAP) function
def mean_average_precision(all_recommendations, all_relevants):
    ap_scores = []
    for recommended, relevant in zip(all_recommendations, all_relevants):
        ap_scores.append(average_precision(recommended, relevant))
    return np.mean(ap_scores)

# Function to check if combined nutritional requirements are met
def check_combined_nutritional_requirements(food1, food2, target_nutrients):
    combined_nutrients = food1[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']] + food2[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']]
    meets_requirements = all(combined_nutrients <= pd.Series(target_nutrients))
    if(combined_nutrients <= pd.Series(target_nutrients)).all():
        return meets_requirements

def check_nutritional_requirements(food, target_nutrients,):
    combined_nutrients = food[['Proteins', 'Carbohydrates', 'Fats', 'Fiber', 'Energy(kcal)']]
    meets_requirements = all(combined_nutrients <= pd.Series(target_nutrients))
    if(combined_nutrients <= pd.Series(target_nutrients)).all():
        return meets_requirements

# Function to get weekly plan and check combined nutritional requirements
def get_weekly_plan(recommendations, associativity_rules, region, target_values):
    weekly_plan = {'breakfast': [], 'lunch': [], 'snacks': [], 'dinner': [], 'appetizers': []}
    
    for meal_type in weekly_plan.keys():
        if recommendations[meal_type]:
            for rec in recommendations[meal_type]:
                food_item = nutrition_data[nutrition_data['Food'] == rec]
                if not food_item.empty:
                    if meal_type == 'lunch' or meal_type == 'dinner':
                        # Ensure that only (1 roti + 1 sabzi) is recommended
                        roti = food_item[food_item['Category'] == 'Roti']
                        sabzi = food_item[food_item['Category'] == 'Sabzi']
                        if not roti.empty and not sabzi.empty:
                            for _, roti_row in roti.iterrows():
                                for _, sabzi_row in sabzi.iterrows():
                                    if check_combined_nutritional_requirements(roti_row, sabzi_row, target_values[meal_type]):
                                        weekly_plan[meal_type].append((roti_row['Food'], sabzi_row['Food']))
                    else:
                        if check_nutritional_requirements(food_item, target_values[meal_type]):
                            weekly_plan[meal_type].append(food_item['Food'].values[0])
    
    return weekly_plan

# Mock functions to demonstrate
def get_recommendations():
    return {
        'breakfast': ['Oats', 'Milk'],
        'lunch': ['Roti', 'Sabzi'],
        'snacks': ['Fruit Salad'],
        'dinner': ['Roti', 'Sabzi'],
        'appetizers': ['Paneer Tikka']
    }

# Mock user input for demonstration
user_allergies = ['peanuts']
user_region_pattern = 'North'
user_category = 'veg'
bmi = 22
gender = 'male'

target_values = set_target_values(bmi, gender)
recommendations = get_recommendations()
filtered_data = filter_dataset(nutrition_data, user_allergies, user_region_pattern, user_category)

# Assuming we have a function to generate relevant items based on user preferences
def get_relevant_items(filtered_data, meal_type):
    # This is a placeholder. Implement according to your actual requirements.
    return filtered_data[filtered_data['Meal_Type'] == meal_type]['Food'].tolist()

all_recommendations = []
all_relevants = []

for meal_type in ['breakfast', 'lunch', 'snacks', 'dinner', 'appetizers']:
    recommended_items = get_weekly_plan(recommendations, associativity_rules, user_region_pattern, target_values)[meal_type]
    relevant_items = get_relevant_items(filtered_data, meal_type)
    
    # Convert to lists of food names
    recommended_foods_list = [item if isinstance(item, str) else item[0] for item in recommended_items]
    relevant_foods_list = relevant_items
    
    all_recommendations.append(recommended_foods_list)
    all_relevants.append(relevant_foods_list)

# Calculate MAP
map_score = mean_average_precision(all_recommendations, all_relevants)
print(f"Mean Average Precision (MAP): {map_score:.4f}")


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().